In [ ]:
import re


def clean_title(title):
    if "wyrok" in title.lower():
        cleaned = clean_court_ruling(title)
    else:
        cleaned = clean_regulation(title)
    return cleaned if len(cleaned) > 45 else title


def clean_court_ruling(title):
    """Handle court rulings specifically"""
    # Extract court name, date, and case number
    court_match = re.match(
        r"^Wyrok\s+(.*?)\s+z\s+dnia\s+(\d+\s+\w+\s+\d{4})\s*r\.\s*sygn\.*(.*?)$",
        title,
        re.IGNORECASE,
    )

    if court_match:
        court = court_match.group(1)
        date = court_match.group(2)
        case_number = court_match.group(3)
        # Format: "Court ruling - case_number (date)"
        cleaned = f"Wyrok {court} - {case_number}"
        return cleaned
    return title


def clean_regulation(title):

    # Extract the authority and the rest of the title
    match = re.match(
        r"^(?:Rozporządzenie|Obwieszczenie)\s+(.*?)\s+z\s+dnia.*?(?:w sprawie|zmieniające)",
        title,
        re.IGNORECASE,
    )
    authority = match.group(1) if match else ""

    # Remove the document type, date, and "w sprawie" phrases
    title = re.sub(r"^.*?(?:z dnia \d+\s+\w+\s+\d{4}\s*r\.\s*)", "", title)
    title = re.sub(
        r"(?:zmieniające\s+rozporządzenie\s+)?w\s+sprawie\s+", "dot. ", title
    )
    title = title.replace(
        "Rzeczypospolitej Polskiej ogłoszenia jednolitego tekstu ustawy", ""
    )

    # Combine authority with cleaned title
    cleaned_title = f"{authority} {title}".strip()

    # Remove code patterns like (PLH120079)
    cleaned_title = re.sub(r"\(\w+\d+\)", "", cleaned_title)

    # Remove extra whitespace
    cleaned_title = re.sub(r"\s+", " ", cleaned_title).strip()

    # Remove specific patterns - now excluding 'Ministra' and 'Marszałka'
    patterns_to_remove = [
        r"Prezesa Rady Ministrów",
        r"Rady Ministrów",
        r"ogłoszenia jednolitego tekstu",
        r"zmieniające rozporządzenie",
    ]
    for pattern in patterns_to_remove:
        cleaned_title = re.sub(pattern, "", cleaned_title)

    # Final cleanup
    cleaned_title = re.sub(r"\s+", " ", cleaned_title).strip()


    return cleaned_title


# Test cases including short titles
test_cases = [
    # Short titles that should be preserved
    "Postanowienie Prezydenta Rzeczypospolitej Polskiej nr 115.7.2023",
    "Ustawa z dnia 9 marca 2023 r. o cudzoziemcach",
    "Wyrok Sądu Najwyższego z dnia 15 marca 2023 r.",
    # Regular longer titles that should be cleaned
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 11 stycznia 2023 r. w sprawie ogłoszenia jednolitego tekstu ustawy o dodatku osłonowym",
    "Wyrok Trybunału Konstytucyjnego z dnia 11 stycznia 2024 r. sygn. akt K 23/23",
    "Rozporządzenie Ministra Zdrowia z dnia 15 lutego 2024 r. w sprawie standardów organizacyjnych (PLH120079)",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 1 marca 2024 r. w sprawie ogłoszenia jednolitego tekstu ustawy o podatku dochodowym",
    "Rozporządzenie Ministra Infrastruktury z dnia 11 września 2024 r. w sprawie zaliczenia dróg do kategorii dróg krajowych",
    "Rozporządzenie Rady Ministrów z dnia 16 września 2024 r. w sprawie wprowadzenia stanu klęski żywiołowej na obszarze części województwa dolnośląskiego, opolskiego oraz śląskiego",
    'Obwieszczenie Ministra Rolnictwa i Rozwoju Wsi z dnia 20 sierpnia 2024 r. w sprawie ogłoszenia jednolitego tekstu rozporządzenia Ministra Rolnictwa i Rozwoju Wsi w sprawie szczegółowych warunków i trybu przyznawania oraz wypłaty pomocy finansowej na operacje typu "Modernizacja gospodarstw rolnych" w ramach poddziałania "Wsparcie inwestycji w gospodarstwach rolnych" objętego Programem Rozwoju Obszarów Wiejskich na lata 2014-2020',
]

# Run tests
for case in test_cases:
    print(f"\nInput  ({len(case)} chars): {case}")
    cleaned = clean_title(case)
    print(f"Output ({len(cleaned)} chars): {cleaned}")

In [ ]:
clean_title("Rozporządzenie Rady Ministrów z dnia 7 sierpnia 2023 r. zmieniające rozporządzenie w sprawie Krajowej Tablicy Przeznaczeń Częstotliwości")

In [ ]:
# load documents from `acts.csv` `
import pandas as pd

acts = pd.read_csv("acts.csv")
acts['title'] = acts['title'].apply(clean_title)

# to csv
acts.to_csv("acts_cleaned.csv", index=False)

# get only 40 rows
acts = pd.read_csv("acts_cleaned.csv", nrows=40)

titles = [
    "Uchwała Senatu Rzeczypospolitej Polskiej z dnia 4 lipca 2024 r. w 35. rocznicę pierwszego posiedzenia odrodzonego Senatu",
    "Uchwała nr 27/2023 Państwowej Komisji Wyborczej z dnia 29 maja 2023 r. zmieniająca uchwałę w sprawie wzorów urn wyborczych",
    "Postanowienie Prezydenta Rzeczypospolitej Polskiej z dnia 30 kwietnia 2024 r. nr 112.21.2024 w sprawie mianowania na stopień oficerski generała",
    "Rozporządzenie Ministra Infrastruktury z dnia 23 grudnia 2022 r. zmieniające rozporządzenie w sprawie warunków eksploatacji lotnisk",
    "Rozporządzenie Ministra Zdrowia z dnia 8 grudnia 2022 r. w sprawie programu pilotażowego badania stóp dzieci i młodzieży",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 7 grudnia 2023 r. w sprawie ogłoszenia jednolitego tekstu ustawy - Prawo telekomunikacyjne",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 1 grudnia 2022 r. w sprawie ogłoszenia jednolitego tekstu ustawy o Polskim Bonie Turystycznym",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 11 października 2023 r. w sprawie ogłoszenia jednolitego tekstu ustawy o pracy na morzu",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 4 listopada 2022 r. w sprawie ogłoszenia jednolitego tekstu ustawy o samorządzie gminnym",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 14 grudnia 2022 r. w sprawie ogłoszenia jednolitego tekstu ustawy o specjalnych strefach ekonomicznych",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 11 marca 2024 r. w sprawie ogłoszenia jednolitego tekstu ustawy o Inspekcji Ochrony Środowiska",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 1 grudnia 2022 r. w sprawie ogłoszenia jednolitego tekstu ustawy o Funduszu Kolejowym",
    "Ustawa z dnia 8 lutego 2023 r. o Planie Strategicznym dla Wspólnej Polityki Rolnej na lata 2023-2027",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 16 listopada 2022 r. w sprawie ogłoszenia jednolitego tekstu ustawy o partnerstwie publiczno-prywatnym",
    "Rozporządzenie Ministra Zdrowia z dnia 5 stycznia 2023 r. zmieniające rozporządzenie w sprawie świadczeń gwarantowanych z zakresu podstawowej opieki zdrowotnej",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 11 marca 2024 r. w sprawie ogłoszenia jednolitego tekstu ustawy o przeciwdziałaniu przemocy domowej",
    "Rozporządzenie Ministra Infrastruktury z dnia 23 grudnia 2022 r. zmieniające rozporządzenie w sprawie klasyfikacji lotnisk i rejestru lotnisk",
    "Postanowienie Prezydenta Rzeczypospolitej Polskiej z dnia 27 kwietnia 2023 r. nr 110.36.2023 w sprawie odwołania Ambasadora Rzeczypospolitej Polskiej",
    "Postanowienie Prezydenta Rzeczypospolitej Polskiej z dnia 27 kwietnia 2023 r. nr 110.38.2023 w sprawie odwołania Ambasadora Rzeczypospolitej Polskiej",
    "Obwieszczenie Marszałka Sejmu Rzeczypospolitej Polskiej z dnia 16 stycznia 2024 r. w sprawie ogłoszenia jednolitego tekstu ustawy o nadzorze nad rynkiem finansowym"
]

cleaned_titles = [clean_title(t) for t in titles]
# there is only title column in the csv file
# documents = acts["title"].tolist() 
documents = cleaned_titles
print(len(documents))
print(documents[:5])

In [5]:
import pandas as pd
from collections import defaultdict
import re
from tqdm import tqdm

def find_common_substrings(df, min_length=20, min_occurrences=10):
    substring_counts = defaultdict(int)
    # Clean and prepare titles
    titles = df['title'].str.strip().tolist()
    total_titles = len(titles)
    
    # First, find all "Rozporządzenie/Obwieszczenie X z dnia" patterns
    skip_pattern = re.compile(r'^(?:Rozporządzenie|Obwieszczenie)\s+.*?\s+z\s+dnia')
    
    print(f"Analyzing substrings in {total_titles} documents...")
    for title in tqdm(titles):
        # Skip administrative part of the title
        skip_match = skip_pattern.match(title)
        if skip_match:
            start_idx = skip_match.end()
            title = title[start_idx:]
        
        # Get all possible substrings of meaningful length
        words = title.split()
        for i in range(len(words)):
            for j in range(i + 1, len(words) + 1):
                substring = ' '.join(words[i:j])
                if len(substring) >= min_length:
                    substring_counts[substring] += 1

    # Filter by minimum occurrences and create DataFrame
    common_substrings = [
        (substr, count, len(substr), round(count/total_titles * 100, 2))
        for substr, count in substring_counts.items()
        if count >= min_occurrences
    ]
    
    # Create DataFrame and sort primarily by occurrences
    result_df = pd.DataFrame(
        common_substrings,
        columns=['substring', 'occurrences', 'length', 'percentage']
    )
    
    # Sort by occurrences (descending) and then by length (descending)
    result_df = result_df.sort_values(
        by=['occurrences', 'length'],
        ascending=[False, False]
    ).reset_index(drop=True)
    
    return result_df, total_titles

def analyze_titles(csv_path, min_length=20, min_occurrences=5):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Find common substrings
    common_substrings_df, total_docs = find_common_substrings(
        df,
        min_length=min_length,
        min_occurrences=min_occurrences
    )
    
    pd.set_option('display.max_colwidth', None)
    print(f"\nAnalyzed {total_docs} documents")
    print(f"Found {len(common_substrings_df)} common substrings")
    print("\nMost common substrings (sorted by number of occurrences):")
    
    # Format the output for better readability
    for idx, row in common_substrings_df.head(20).iterrows():
        print("\n" + "="*80)
        print(f"#{idx + 1}: Occurs in {row['occurrences']} documents ({row['percentage']}%)")
        print(f"Length: {row['length']} characters")
        print(f"Substring: {row['substring']}")
    
    # Summary statistics
    print("\n" + "="*80)
    print("\nOccurrence Statistics:")
    print(f"Most frequent: {common_substrings_df['occurrences'].max()} occurrences")
    print(f"Median occurrences: {common_substrings_df['occurrences'].median()}")
    print(f"Mean occurrences: {common_substrings_df['occurrences'].mean():.2f}")
    
    return common_substrings_df

# Usage:
# results_df = analyze_titles('Acts.csv', min_length=20, min_occurrences=5)

In [ ]:
analyze_titles('acts_cleaned.csv')

In [ ]:
from __future__ import annotations
import os
from pathlib import Path
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import vertexai

credentials_path = Path().cwd() / "sejm-stats-439117-39efc9d2f8b8.json"
print(credentials_path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(credentials_path)


vertexai.init(project="sejm-stats-439117")



def embed_text(texts: list) -> list[list[float]]:
    dimensionality = 512
    task = "RETRIEVAL_DOCUMENT"


    model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")


    inputs = [TextEmbeddingInput(text, task) for text in texts]


    kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}


    embeddings = model.get_embeddings(inputs, **kwargs)

    return [embedding.values for embedding in embeddings]

In [ ]:
import numpy as np
embeddings = embed_text(documents)
print(len(embeddings[0]))
embeddings = np.array(embeddings)

In [ ]:
import ast
em_df =pd.read_csv("embeded.csv")
em_df
titles= em_df["title"]
def string_to_array(embedding_str):
    # Convert string representation of list to actual list
    try:
        # Remove any whitespace and convert to list
        embedding_list = ast.literal_eval(embedding_str)
        return np.array(embedding_list)
    except:
        return None

# Convert embeddings to numpy arrays
embeddings = em_df["embedding"].apply(string_to_array)
embeddings =  np.vstack(embeddings.to_numpy())
embeddings

In [ ]:
# test = "specjalnych strefach ekonomicznychna"
test = "Obwieszczenie Ministra Sprawiedliwości z dnia 11 października 2024 r. w sprawie ogłoszenia jednolitego tekstu rozporządzenia Ministra Sprawiedliwości w sprawie organizacji i przebiegu aplikacji notarialnej"
test = "organizacji i przebiegu aplikacji notarialnej"
phrase_embedding = embed_text([test])
# phrase_embedding = np.array(phrase_embedding, dtype=np.float64)
phrase_embedding

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(phrase_embedding, embeddings)

# Get the indices of the top 5 most similar documents
top_5_indices = similarities[0].argsort()[-5:][::-1]

# Print the top 5 most similar documents and their similarity scores
for index in top_5_indices:
    print(titles[index])
    print(similarities[0][index])